In [15]:
import pandas as pd
import string
import numpy as np
import os
print(os.getcwd())

d:\OneDrive - MNSCU\Research\2023_AI_Text_Classification\AI_Generated_Text_Classfier


In [16]:
df1 = pd.read_csv('./data/features_GT.csv')
df1.sort_values(by='indexes', inplace=True)
print(df1.shape)

(3974, 50786)


In [17]:
df2 = pd.read_csv('./data/features_ChatGPT.csv')
df2.sort_values(by='indexes', inplace=True)
print(df2.shape)

(4557, 50786)


In [18]:
df1_filtered = df1.loc[df1['indexes'].isin(df2['indexes'])]
print(df1_filtered.shape)

(3643, 50786)


In [19]:
df2_filtered = df2.loc[df2['indexes'].isin(df1['indexes'])]
print(df2_filtered.shape)

(3643, 50786)


In [28]:
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler

x1 = df1_filtered.iloc[:, 3:44]
x2 = df2_filtered.iloc[:, 3:44]
x_merged = pd.concat([x1, x2], ignore_index = True, sort = False)
scaler = StandardScaler().fit(x_merged)
print('scaler mean : ', scaler.mean_)
print('scaler scale: ', scaler.scale_)
x1 = scaler.transform(x1)
x2 = scaler.transform(x2)

similarities = []
for i in tqdm(range(df1_filtered.shape[0])):
    try:
        gt_features = x1[i]
        #print(gt_features.shape)
        chatgpt_features = x2[i]
        #print(chatgpt_features.shape)
        similarity_score = cosine_similarity([gt_features], [chatgpt_features])
        #print('row ', i, ': score=', similarity_score)
        similarities.append(similarity_score)
    except Exception as e:
        similarity_score = 0
        similarities.append(similarity_score)
        print('Cosine error at row ', i, str(e))
        break
similarities = np.reshape(similarities, (-1,1))
similarities = pd.DataFrame(similarities, columns=['Cosine similarity'])
similarities.to_csv('./data/wiki_features_similarity_no_tfidf.csv', index=False)

scaler mean :  [1.87782460e+02 1.18323511e+03 6.23811837e+00 3.40145484e+01
 3.42327752e+01 2.56848751e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.57260500e+00
 9.99383253e+00 2.87756974e+01 1.14008870e+01 9.14104216e+00
 8.02926679e+00 1.05990390e+01 1.23704912e+01 6.05840005e+00
 2.12215207e+01 4.19854151e-02 7.58861542e-02 7.01157543e-02
 4.16464565e-02 4.27094388e-02 4.37071065e-02 4.22704968e-02
 1.05933527e-01 4.18976376e-02 6.57322406e-02 4.18208612e-02
 4.16028739e-02 4.94075877e-02 4.29343990e-02 4.22354436e-02
 4.25470375e-02 4.17126971e-02 4.16858150e-02 4.21435145e-02
 4.20255434e-02]
scaler scale:  [1.70351716e+02 1.10299942e+03 6.64007049e-01 4.62950094e+01
 4.07394546e+01 5.33144285e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 2.73225744e+01
 8.60905340e+00 2.61316823e+01 9.40943331e+00 5.95514268e+00
 5.07847339e+00 1.02929671e+01 1.30756644e+01 4.38914480e+00
 2.74447226e+01 2.34595596e-02 7.15685

100%|██████████| 3643/3643 [00:00<00:00, 15209.88it/s]


In [30]:
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

x1 = df1_filtered.iloc[:, 3:]
x2 = df2_filtered.iloc[:, 3:]
x_merged = pd.concat([x1, x2], ignore_index = True, sort = False)
scaler = StandardScaler().fit(x_merged)
print('scaler mean : ', scaler.mean_)
print('scaler scale: ', scaler.scale_)
x1 = scaler.transform(x1)
x2 = scaler.transform(x2)

similarities_full = []
for i in tqdm(range(df1_filtered.shape[0])):
    try:
        gt_features = x1[i]
        #print(gt_features.shape)
        chatgpt_features = x2[i]
        #print(chatgpt_features.shape)
        similarity_score = cosine_similarity([gt_features], [chatgpt_features])
        #print('row ', i, ': score=', similarity_score)
        similarities_full.append(similarity_score)
    except Exception as e:
        similarity_score = 0
        similarities_full.append(similarity_score)
        print('Cosine error at row ', i, str(e))
        break
similarities_full = np.reshape(similarities_full, (-1,1))
similarities_full = pd.DataFrame(similarities_full, columns=['Cosine similarity'])
similarities_full.to_csv('./data/wiki_features_similarity_with_tfidf.csv', index=False)

scaler mean :  [1.87782460e+02 1.18323511e+03 6.23811837e+00 ... 6.03005484e-04
 7.79116701e-04 1.23469147e-03]
scaler scale:  [1.70351716e+02 1.10299942e+03 6.64007049e-01 ... 1.00323426e-02
 1.01743734e-02 1.94236586e-02]


100%|██████████| 3643/3643 [00:05<00:00, 675.90it/s]


In [32]:
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA

#########################################################################
x1 = df1_filtered.iloc[:, 3:]
x2 = df2_filtered.iloc[:, 3:]
x_merged = pd.concat([x1, x2], ignore_index = True, sort = False)
scaler = StandardScaler().fit(x_merged)
print('scaler mean : ', scaler.mean_)
print('scaler scale: ', scaler.scale_)
x1 = scaler.transform(x1)
x2 = scaler.transform(x2)
#########################################################################
pca1 = PCA(n_components=1024)
x1 = pca1.fit_transform(x1)
print('pca.explained_variance_ratio_', pca1.explained_variance_ratio_)
print('pca.singular_values_', pca1.singular_values_)
pca2 = PCA(n_components=1024)
x2 = pca2.fit_transform(x2)
print('pca.explained_variance_ratio_', pca2.explained_variance_ratio_)
print('pca.singular_values_', pca2.singular_values_)
print(x1.shape, x2.shape)

similarities_full = []
for i in tqdm(range(df1_filtered.shape[0])):
    try:
        gt_features = x1[i]
        #print(gt_features.shape)
        chatgpt_features = x2[i]
        #print(chatgpt_features.shape)
        similarity_score = cosine_similarity([gt_features], [chatgpt_features])
        #print('row ', i, ': score=', similarity_score)
        similarities_full.append(similarity_score)
    except Exception as e:
        similarity_score = 0
        similarities_full.append(similarity_score)
        print('Cosine error at row ', i, str(e))
        break
similarities_full = np.reshape(similarities_full, (-1,1))
similarities_full = pd.DataFrame(similarities_full, columns=['Cosine similarity'])
similarities_full.to_csv('./data/wiki_features_similarity_with_tfidf_pca.csv', index=False)

scaler mean :  [1.87782460e+02 1.18323511e+03 6.23811837e+00 ... 6.03005484e-04
 7.79116701e-04 1.23469147e-03]
scaler scale:  [1.70351716e+02 1.10299942e+03 6.64007049e-01 ... 1.00323426e-02
 1.01743734e-02 1.94236586e-02]
pca.explained_variance_ratio_ [0.00673364 0.00531963 0.00425419 ... 0.00027464 0.00027417 0.0002732 ]
pca.singular_values_ [1186.3290704  1054.43736588  942.95007053 ...  239.58744016  239.38212764
  238.95725349]
pca.explained_variance_ratio_ [0.00357574 0.00307535 0.00260167 ... 0.000295   0.00029395 0.00029273]
pca.singular_values_ [597.85098647 554.44356862 509.95984971 ... 171.7191562  171.41418255
 171.05767835]
(3643, 1024) (3643, 1024)


100%|██████████| 3643/3643 [00:00<00:00, 13134.45it/s]
